## Creating a DataFrame

In the [cc_scraping.ipynb](Documents/Coding/Notebooks/tutorials/cc_scraping.ipynb) demo, we extracted data already from a pretty friendly F1 table.

What if we didn't have that? 

In this self-assigned exercise I will extract simple data from all the matches played in the EURO 2020 from the [Flash Score](https://www.flashscore.com/football/europe/euro-2020/results/) website. We will form a dataframe from the following info
1. Date
2. Stage of Competition
3. Home Team
4. Away Team
5. Score
6. Penalty Score

All using Requests, BeautifulSoup, then Pandas

In [232]:
import requests
from bs4 import BeautifulSoup

euro_url = "https://terrikon.com/en/euro-2020"
e_page = requests.get(euro_url)
e_html = e_page.text
e_soup = BeautifulSoup(e_html)

e_tables = e_soup.find("div", class_="maincol")
e_headers = e_tables.find_all("h2")
e_SoC_names = ["".join(header.text.split("Euro-2020."))[1:] for header in e_headers]

##### Sections HTML by Matchtype
*i.e Final, Semi-Finals . . . Group F*

In [233]:
e_sections = []
for cnt, header in enumerate(e_headers):
    tracker = header.next_sibling
    cur_html = ""
    next_header = False
    try:
        next_header = e_headers[cnt +1]
    except:
        while tracker: 
            cur_html = cur_html + str(tracker)
            tracker = tracker.next_sibling
        if cur_html:
            e_sections.append(BeautifulSoup(cur_html).div)
        continue
    while next_header and tracker != next_header and tracker:
            cur_html = cur_html + str(tracker)
            tracker = tracker.next_sibling
    if cur_html:
            e_sections.append(BeautifulSoup(cur_html).div)

##### Adds the Match type by the relevant Stage
*i.e Reporting data for a game in the final*



In [239]:
stage_tester = e_sections[0]
s_table = stage_tester.select("table.gameresult")[0]